For the GloVe Embeddings Model

In [ ]:
pip install shap


In [ ]:
import shap
import numpy as np

# Assuming `model` is your trained LSTM model from the GloVe code
explainer = shap.DeepExplainer(model, X_train)
shap_values = explainer.shap_values(X_test)

# Plot the SHAP values for a single prediction
shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0][0], X_test[0])

# To visualize overall feature importance:
shap.summary_plot(shap_values[0], X_test)

In [ ]:
pip install lime

In [ ]:
from lime.lime_text import LimeTextExplainer
import numpy as np

explainer = LimeTextExplainer(class_names=["Non-MI Incident", "MI Incident"])

def predict_proba(texts):
    sequences = tokenizer.texts_to_sequences(texts)
    padded = pad_sequences(sequences, maxlen=maxlen)
    return model.predict(padded)

# Explain a single instance
i = 0  # index of the test example you want to explain
exp = explainer.explain_instance(X_test[i], predict_proba, num_features=10)
exp.show_in_notebook()

For the SciBERT Model

In [ ]:
import shap
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased")
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

# Create a SHAP explainer for Transformer models
explainer = shap.Explainer(model, masker=tokenizer)

# Explain a single prediction
shap_values = explainer(X_test[:10])  # Explain the first 10 samples
shap.plots.text(shap_values[0])  # Show explanation for the first text


In [ ]:
pip install captum

import torch
from captum.attr import IntegratedGradients
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased")
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

# Prepare the data
input_text = "Example text to interpret"
input_ids = tokenizer.encode(input_text, return_tensors='pt')
input_ids = input_ids.to('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize Integrated Gradients
ig = IntegratedGradients(model)

# Compute attributions
attributions, delta = ig.attribute(inputs=input_ids, target=0, return_convergence_delta=True)

# Visualize attributions
attributions_sum = attributions.sum(dim=-1).squeeze(0).cpu().detach().numpy()
tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze(0).cpu().detach().numpy())

# Print tokens with their attributions
for token, attr in zip(tokens, attributions_sum):
    print(f"{token}: {attr:.4f}")
